In [ ]:
import pandas as pd
import os

In [ ]:
!kaggle competitions download -c fake-news

  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 15.3MB/s]
 53% 5.00M/9.42M [00:00<00:00, 25.8MB/s]
100% 9.42M/9.42M [00:00<00:00, 37.4MB/s]
 67% 25.0M/37.0M [00:00<00:00, 22.1MB/s]
100% 37.0M/37.0M [00:01<00:00, 36.4MB/s]


In [ ]:
!unzip "/content/train.csv.zip"

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
df = pd.read_csv("/content/train.csv")

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df = df.dropna()

In [ ]:
X= df.drop('label', axis= 1)

In [ ]:
y = df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
#Vocabulary size
voc_size = 5000

**One Hot Representation**

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace = True)
# As we had dropped NaN values from X

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
corpus = []

for i in range(0, len(messages)):  
  review = re.sub("[^a-zA-Z]", " ", messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
  review = " ".join(review)
  corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]

In [ ]:
onehot_rep

**Embedding Representation**

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, maxlen=sent_length, padding = "pre")

In [ ]:
embedded_docs

array([[   0,    0,    0, ..., 4321, 2286, 4517],
       [   0,    0,    0, ..., 1002,  988,  829],
       [   0,    0,    0, ..., 4616,  566, 3177],
       ...,
       [   0,    0,    0, ...,  421, 3280,  498],
       [   0,    0,    0, ..., 2965, 4191, 2518],
       [   0,    0,    0, ..., 2631, 4822, 3802]], dtype=int32)

**Model Creation**

In [ ]:
embedding_vector_features = 40

model= Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss= "binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [ ]:
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_4 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final , test_size = 0.3, random_state = 42)

**Model Training**

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)

Epoch 1/20
400/400 [==============================] - 8s 19ms/step - loss: 0.1384 - accuracy: 0.9474 - val_loss: 0.1943 - val_accuracy: 0.9180
Epoch 2/20
400/400 [==============================] - 8s 19ms/step - loss: 0.1003 - accuracy: 0.9642 - val_loss: 0.2277 - val_accuracy: 0.9041
Epoch 3/20
400/400 [==============================] - 8s 19ms/step - loss: 0.0658 - accuracy: 0.9766 - val_loss: 0.2669 - val_accuracy: 0.9103
Epoch 4/20
400/400 [==============================] - 8s 19ms/step - loss: 0.0460 - accuracy: 0.9841 - val_loss: 0.2854 - val_accuracy: 0.9114
Epoch 5/20
400/400 [==============================] - 8s 20ms/step - loss: 0.0319 - accuracy: 0.9896 - val_loss: 0.3107 - val_accuracy: 0.9134
Epoch 6/20
400/400 [==============================] - 8s 19ms/step - loss: 0.0236 - accuracy: 0.9927 - val_loss: 0.3964 - val_accuracy: 0.9127
Epoch 7/20
400/400 [==============================] - 8s 20ms/step - loss: 0.0145 - accuracy: 0.9953 - val_loss: 0.4911 - val_accuracy: 0.9089

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

0.907036091870215